### Extra Bokeh

In [ ]:
import pandas as pd

from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook


In [ ]:
output_notebook()

In [ ]:
url_users:str = r"https://raw.githubusercontent.com/Andru-1987/csv_files_ds/refs/heads/main/Mall_Customers.csv"
df:pd.DataFrame = pd.read_csv(url_users, sep=",", encoding="utf8")

df.head()


### Grafico de Lineas


In [ ]:
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Category10
from bokeh.transform import factor_cmap

In [ ]:
# Agrupar por edad y calcular estadísticas de ingresos
age_income = df.groupby('Age')['Annual Income (k$)'].agg(['mean', 'median', 'std']).reset_index()
age_income = age_income.sort_values('Age')

age_income.head()

In [ ]:
# Crear ColumnDataSource
source_line = ColumnDataSource(age_income)

# Crear figura para gráfico de líneas
p_line = figure(
    width=800, 
    height=400,
    title="Distribución de Ingresos Anuales por Edad",
    x_axis_label='Edad',
    y_axis_label='Ingreso Anual (k$)',
    tools="pan,wheel_zoom,box_zoom,reset,save"
)


# Añadir múltiples líneas
p_line.line('Age', 'mean', source=source_line, 
           line_width=3, color=Category10[3][0], 
           legend_label='Ingreso Promedio', alpha=0.8)

p_line.line('Age', 'median', source=source_line, 
           line_width=3, color=Category10[3][1], 
           legend_label='Mediana de Ingresos', alpha=0.8)

p_line.line('Age', 'std', source=source_line, 
           line_width=3, color=Category10[3][2], 
           legend_label='Desvio St de Ingresos', alpha=0.8)

# Configurar tooltips
hover_line = HoverTool()
hover_line.tooltips = [
    ("Edad", "@Age"),
    ("Ingreso Promedio", "@mean{0.0} k$"),
    ("Mediana", "@median{0.0} k$"),
    ("Desviación Std", "@std{0.0} k$")
]
p_line.add_tools(hover_line)

# # Personalizar
# p_line.legend.location = "top_left"
# p_line.legend.click_policy = "hide"
# p_line.xgrid.grid_line_color = None
# p_line.ygrid.grid_line_alpha = 0.3

show(p_line)

### Grafico de ScatterPlot

In [ ]:
# Scatterplot con color diferenciado por género
source_gender = ColumnDataSource(df)

p_gender = figure(
    width=600, 
    height=400,
    title="Edad vs Ingreso Anual - Coloreado por Género",
    x_axis_label='Edad (Años)',
    y_axis_label='Ingreso Anual (k$)',
    tools="pan,wheel_zoom,box_zoom,reset,save"
)

# Mapeo de colores por género
color_map = {'Male': 'blue', 'Female': 'red'}

# Crear scatterplot con colores por género
p_gender.scatter(
    x='Age', 
    y='Annual Income (k$)', 
    source=source_gender,
    size=8,
    color=factor_cmap('Gender', palette=[color_map['Male'], color_map['Female']], 
                     factors=list(color_map.keys())),
    alpha=0.7,
    line_color='white',
    line_width=0.5,
    legend_field='Gender'
)

# # Tooltips
# hover_gender = HoverTool()
# hover_gender.tooltips = [
#     ("Edad", "@Age años"),
#     ("Ingreso Anual", "@{Annual Income (k$)} k$"),
#     ("Género", "@Gender"),
#     ("Spending Score", "@{Spending Score (1-100)}")
# ]
# p_gender.add_tools(hover_gender)

# # Personalizar leyenda
# p_gender.legend.location = "top_right"
# p_gender.legend.title = "Género"
# p_gender.legend.click_policy = "hide"

show(p_gender)

### HeatMap

In [ ]:
import numpy as np

In [ ]:
from bokeh.models import ColumnDataSource, HoverTool, LinearColorMapper, ColorBar
from bokeh.palettes import Viridis256, Blues256

In [ ]:
# Calcular matriz de correlación
corr_matrix = df[['Age', 'Annual Income (k$)', 'Spending Score (1-100)']].corr()
variables = corr_matrix.columns.tolist()

# Preparar datos correctamente
data = {
    'x': [],
    'y': [], 
    'valores': [],
    'x_label': [],
    'y_label': []
}

for i in range(len(variables)):
    for j in range(len(variables)):
        data['x'].append(i)
        data['y'].append(j)
        data['valores'].append(corr_matrix.iloc[j, i])  # [j,i] para orientación correcta
        data['x_label'].append(variables[i])
        data['y_label'].append(variables[j])

source = ColumnDataSource(data)

# Crear figura
p = figure(
    width=800,
    height=800,
    title="Heatmap Simple - Correlaciones",
    x_range=variables,
    y_range=list(reversed(variables)),
    tools="hover,save"
)

# Mapper de colores
color_mapper = LinearColorMapper(palette=Blues256, low=-1, high=1)

# Crear rectángulos
p.rect(
    x='x_label',
    y='y_label',
    width=1,
    height=1,
    source=source,
    fill_color={'field': 'valores', 'transform': color_mapper},
    line_color='white'
)

# Barra de colores
color_bar = ColorBar(color_mapper=color_mapper, location=(0, 0))
p.add_layout(color_bar, 'right')

# Tooltip
hover = p.select_one(HoverTool)
hover.tooltips = [
    ("Variables", "@x_label vs @y_label"),
    ("Correlación", "@valores{0.00}")
]

# Opciones estéticas
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "12pt"

# 🔹 Rotar etiquetas del eje X 45 grados
p.xaxis.major_label_orientation = 45  # En radianes si querés más fino, ej: np.pi/4

show(p)


### Radar

In [ ]:
import math
from bokeh.plotting import figure, show
from bokeh.models import Legend
import pandas as pd



In [ ]:
def polar(angulos, valores):
    """Convierte ángulos y valores en coordenadas cartesianas (x, y)."""
    x = [math.cos(a) * v for a, v in zip(angulos, valores)]
    y = [math.sin(a) * v for a, v in zip(angulos, valores)]
    return x, y


def crear_radar_simple(df):
    # --- 1. Calcular promedios ---
    columnas = ['Age', 'Annual Income (k$)', 'Spending Score (1-100)']
    categorias = ['Edad', 'Ingreso', 'Spending Score']
    stats = df.groupby('Gender')[columnas].mean()

    # --- 2. Preparar datos ---
    female_vals = stats.loc['Female'].values
    male_vals = stats.loc['Male'].values

    # Cerrar el círculo
    female_vals = list(female_vals) + [female_vals[0]]
    male_vals = list(male_vals) + [male_vals[0]]

    # --- 3. Normalizar ---
    max_vals = df[columnas].max()
    female_norm = (stats.loc['Female'] / max_vals).tolist()
    male_norm = (stats.loc['Male'] / max_vals).tolist()
    female_norm += [female_norm[0]]
    male_norm += [male_norm[0]]

    # --- 4. Calcular coordenadas ---
    angulos = [n / len(categorias) * 2 * math.pi for n in range(len(categorias))]
    angulos += [angulos[0]]

    x_f, y_f = polar(angulos, female_norm)
    x_m, y_m = polar(angulos, male_norm)

    # --- 5. Crear ColumnDataSource ---
    source_f = ColumnDataSource(data=dict(
        x=x_f,
        y=y_f,
        categoria=categorias + [categorias[0]],
        valor_original=female_vals,
        valor_norm=female_norm
    ))

    source_m = ColumnDataSource(data=dict(
        x=x_m,
        y=y_m,
        categoria=categorias + [categorias[0]],
        valor_original=male_vals,
        valor_norm=male_norm
    ))

    # --- 6. Crear figura ---
    p = figure(width=500, height=500, title="Radar Chart - Female vs Male",
               x_range=(-1.5, 1.5), y_range=(-1.5, 1.5),
               toolbar_location=None, tools="save,pan,reset")

    # Líneas radiales + etiquetas
    for i, cat in enumerate(categorias):
        a = angulos[i]
        p.line([0, math.cos(a)], [0, math.sin(a)], color='gray', alpha=0.3)
        p.text(math.cos(a) * 1.2, math.sin(a) * 1.2, text=[cat],
               text_align='center', text_baseline='middle')

    # --- Círculos de referencia ---
    for r in [0.25, 0.5, 0.75, 1]:
        p.annulus(
            x=0, y=0,
            inner_radius=r - 0.001,
            outer_radius=r,
            line_color='gray',
            fill_color=None,
            alpha=0.2
        )

    # --- 7. Dibujar áreas + puntos ---
    r1 = p.patch('x', 'y', source=source_f, color='red', alpha=0.4)
    r2 = p.patch('x', 'y', source=source_m, color='blue', alpha=0.4)

    p.line('x', 'y', source=source_f, color='red', line_width=2)
    p.line('x', 'y', source=source_m, color='blue', line_width=2)

    p.scatter('x', 'y', source=source_f, color='red', size=6)
    p.scatter('x', 'y', source=source_m, color='blue', size=6)

    # --- 8. Hover interactividad ---
    hover = HoverTool(
        tooltips=[
            ("Categoría", "@categoria"),
            ("Valor original", "@valor_original{0.0}"),
            ("Valor normalizado", "@valor_norm{0.00}"),
            ("(x,y)", "(@x{0.00}, @y{0.00})")
        ],
        mode='mouse'
    )
    p.add_tools(hover)

    # --- 9. Leyenda ---
    legend = Legend(items=[("Female", [r1]), ("Male", [r2])], location="top_right")
    p.add_layout(legend)

    return p


In [ ]:
radar_simple = crear_radar_simple(df)
show(radar_simple)


### Bar

In [ ]:
conteo_genero = df['Gender'].value_counts()

source = ColumnDataSource(data=dict(
    genero=list(conteo_genero.index),
    conteo=conteo_genero.values,
    porcentaje=(conteo_genero.values / len(df) * 100).round(1),
    color=['pink', 'blue']
))

p = figure(
    width=400,
    height=400,
    title="Clientes por Género",
    x_range=list(conteo_genero.index)
)

p.vbar(
    x='genero',
    top='conteo',
    width=0.5,
    color='color',
    source=source
)

hover = HoverTool()
hover.tooltips = [
    ("Género", "@genero"),
    ("Clientes", "@conteo"),
    ("Porcentaje", "@porcentaje%")
]

p.add_tools(hover)

show(p)